In [128]:
import spacy
import re

from typing import List, Tuple
from nltk.corpus import stopwords as StopWords

In [129]:
nlp = spacy.load("pt_core_news_sm")

In [242]:
REGEX_TO_SPLIT_DOCUMENTS = "(O|A)+\s+Sr(\.|\.º|\.ª)\s+([A-zÀ-ú]|\s*)+(\(.*\))?: —"


deputies_docs_unprocessed = {}
documents_unprocessed_idx = {}
documents_to_deputies = {}

doc_idx = 0
did_first_match = False

with open("resources/example_of_parlamentar_discussion/darl14sl02n014.txt") as file:
    
    first_line = next(file)
    DATE_SECTION_REGEX = "(?i)\d+ de (\w+) de \d{4}"
    romanic_number = "(?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3})"
    SERIES_SECTION_REGEX = romanic_number + " (Série|SÉRIE) — (Número|NÚMERO) \d{1,3}"
    
    
    pattern = re.compile(REGEX_TO_SPLIT_DOCUMENTS)
    date_section_pattern = re.compile(DATE_SECTION_REGEX)
    series_section_pattern = re.compile(SERIES_SECTION_REGEX)
    numberic_pattern = re.compile("\d+")
    
    current_docs = ""
    current_deputy = None
    
    for line in file:
        date_section_match = date_section_pattern.search(line)
        series_section_match = series_section_pattern.search(line)
        if date_section_match is not None or series_section_match is not None:
            #we are in a section, let's consume until a number appear
            line_is_page_number = False
            while not line_is_page_number:
                #check if line is number
                #if it is, then line_is_page_number = True
                line = next(file)
                numeric_match = numberic_pattern.search(line)
                if numeric_match is not None:
                    line_is_page_number = True
                    line = next(file)
        match = pattern.search(line)
        if match is not None:
            #a new document
            #is this the first one? if it is, then we already consumed the summary section
            if current_deputy is not None:
                #save current document
                documents_unprocessed_idx[doc_idx] = current_docs
                if current_deputy not in deputies_docs_unprocessed:
                    deputies_docs_unprocessed[current_deputy] = []
                deputies_docs_unprocessed[current_deputy].append(doc_idx)
                documents_to_deputies[doc_idx] = current_deputy
                doc_idx += 1
            #docs stored. start processing new one
            current_deputy = match.group()[0:-3]
            current_docs = line.replace(current_deputy, '')
        else:
            current_docs += line
    if current_deputy is not None:
                #save current document
                documents_unprocessed_idx[doc_idx] = re.sub(' +',' ',current_docs.replace('\n',' '))
                if current_deputy not in deputies_docs_unprocessed:
                    deputies_docs_unprocessed[current_deputy] = []
                deputies_docs_unprocessed[current_deputy].append(doc_idx)
                documents_to_deputies[doc_idx] = current_deputy
                doc_idx += 1

                

In [243]:
pt_stop_words = StopWords.words('portuguese')

def parse_entities(document: str) -> List[str]:
    nlp_doc = nlp(document)
    return [re.sub(' +', ' ', ent.text.replace("\n"," ")) for ent in nlp_doc.ents]
    

In [244]:
documents_processed_entities = {}

for idx in documents_unprocessed_idx:
    documents_processed_entities[idx] = parse_entities(documents_unprocessed_idx[idx])

entities_corpus = list(documents_processed_entities.values())

In [245]:
docs_to_be_removed = []
for idx in documents_processed_entities.keys():
    if len(documents_processed_entities[idx]) == 0:
        docs_to_be_removed.append(idx)
        deputies_docs_unprocessed[documents_to_deputies[idx]].remove(idx)
        del documents_to_deputies[idx]

for el in sorted(docs_to_be_removed, reverse=True):
    del documents_processed_entities[el]
    
entities_corpus = list(documents_processed_entities.values())

In [246]:
entities_corpus_idf_search = list(map(set, entities_corpus))

vocab = set()

for doc in entities_corpus:
    vocab = vocab.union(set(doc))  

print(f"My vocabolary size is {len(vocab)}")

My vocabolary size is 444


In [247]:
import math

idfvocab = {}

def idf(term, corpus):
    cnt =  sum([1 if term in doc else 0 for doc in corpus])
    return math.log10( len(corpus) / cnt )

for term in vocab:
    term_idf = idf(term, entities_corpus_idf_search)
    idfvocab[term] = term_idf

In [248]:
import numpy as np

idfvocab_it = [(el[0],el[1]) for el in idfvocab.items()]

aux = np.array( idfvocab_it )
low = float( min( aux[:,1] ) )
high = float( max( aux[:,1] ) )

print(f"Min is {low} and max is {high}")

Min is 0.47921424422550624 and max is 2.3180633349627615


In [249]:
vc = np.array(idfvocab_it) #a matrix, with column 0 being terms and column 1 being idf
vc_terms = vc[:,0]

In [250]:
def normTFx(term,doc):
    return doc.count(term)/len(doc)

def tfidfmat(corpus, tl, idfvocab) :
    mat =[]
    for term in tl :
        idft = idfvocab[term]
        row = []
        for doc in corpus:
            tft = normTFx(term,doc)
            tf_idf_term_document = tft*idft
            row.append(tf_idf_term_document)
        mat.append(row)
    return mat    
            
    

tfidf_matrix = tfidfmat(entities_corpus, vc_terms, idfvocab) 
tfidf_matrix_np = np.array(tfidf_matrix)

In [251]:
tfidf_matrix_np.shape

(444, 208)

In [260]:
def print_entities(idx_entities_scored: List[Tuple[int, float]]):
    for idx, score in idx_entities_scored:
        print(f"{vc_terms[idx]} - {round(score,3)}")

def column_in_tfidf_of_document_with_index(document_index):
    return list(documents_processed_entities.keys()).index(document_index)
        
def print_info_for_document(document_idx):
    print(documents_to_deputies[document_idx])
    column_idx = column_in_tfidf_of_document_with_index(document_idx)
    print_entities(list(filter(lambda x: x[1] > 0, sorted(enumerate(tfidf_matrix_np[:,column_idx].tolist()), key=lambda x: x[1], reverse=True))))
        
print_info_for_document(deputies_docs_unprocessed["O Sr. André Ventura (CH)"][12])

O Sr. André Ventura (CH)
Sr. Deputado João Cotrim de Figueiredo - 0.92
Sr. Presidente - 0.24


In [178]:
assert len(list(documents_processed_entities.keys())) == len(documents_to_deputies)

In [259]:
documents_unprocessed_idx[deputies_docs_unprocessed["O Sr. André Ventura (CH)"][12]]

': — Sr. Presidente, eu e o Sr. Deputado João Cotrim de Figueiredo temos \n\npermanentemente a indicação de que temos de usar máscara… \n\n\n\n'

In [ ]:
# Talvez usar tf-idf mas com lemmas em vez de stemmer?

In [12]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

A A DET DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art det X True False
Apple Apple PROPN PROPN__Gender=Fem|Number=Sing nsubj Xxxxx True False
está estar VERB VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin ROOT xxxx True True
a o SCONJ SCONJ mark x True False
ponderar ponderar VERB VERB__VerbForm=Inf xcomp xxxx True False
comprar comprar VERB VERB__VerbForm=Inf xcomp xxxx True False
a o DET DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art det x True False
startup startup NOUN NOUN__Gender=Fem|Number=Sing obj xxxx True False
do do DET ADP_DET__Definite=Def|Gender=Masc|Number=Sing|PronType=Art case xx True True
Reino Reino PROPN PROPN__Gender=Masc|Number=Sing nmod Xxxxx True False
Unido Unido PROPN PROPN__Number=Sing flat:name Xxxxx True False
por por ADP ADP case xxx True True
1 1 NUM NUM__NumType=Card obl d False False
milhão milhão NUM NUM__NumType=Card flat xxxx True False
de de ADP ADP case xx True True
dolares dolares NOUN NOUN__Gender=Masc|Number=Plur nmod

In [11]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 2 7 ORG
Reino Unido 45 56 LOC


In [17]:
def print_entities(doc):
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

def print_tokens(doc):
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [19]:
doc_test = nlp("O Presidente declarou aberta a sessão às 15 horas e 2 minutos.")
print_entities(doc_test)

Presidente 2 12 MISC


In [27]:
doc = nlp("Sr. Presidente, passo a responder a este lote de três perguntas, as quais \
\
agradecemos, como é óbvio. \
\
Sr. Deputado José Luís Ferreira, obviamente que o Partido Socialista considera a agricultura familiar como \
\
um fator de desenvolvimento dos territórios de baixa densidade e olha para esta vertente da agricultura como \
\
aquela que é a mais genuína das formas de agricultura e que tem de ser, obviamente, valorizada no contexto \
\
do panorama nacional. \
\
Aplausos do PS. \
\
Sr. Deputado João Dias, se me permite, este é que é o ponto: não podemos colocar uma das questões que \
\
é para nós importante para desenvolver os territórios de baixa densidade como se fosse a única, só e ela mesma \
\
a responsável pelo desenvolvimento económico, porque há outras formas de agricultura, há outras formas de \
\
indústria, há outra capacidade produtiva que também tem de ser gerada no interior. Há sofisticação que tem \
\
ser gerada no interior, tem de ser potenciada no interior e não podemos olhar só para metade do caminho, \
\
porque, se só olharmos para metade do caminho, também defraudamos as expectativas dessas pessoas. \
\
Sr. Deputado: serviços públicos? Falemos dos serviços públicos que este Governo e o anterior reabriram. \
\
Não preciso de os elencar, porque o Sr. Deputado também faz parte de uma solução de apoio a um Governo \
\
que permitiu abrir serviços públicos no interior. ")

print_entities(doc)
print("*"*10)
print_tokens(doc)

Sr. Presidente 0 14 PER
Sr. Deputado José Luís Ferreira 101 132 PER
Partido Socialista 151 169 ORG
PS 458 460 ORG
Sr. Deputado João Dias 462 484 PER
Sr. 1091 1094 PER
Governo 1163 1170 LOC
Sr. Deputado 1231 1243 PER
Governo 1290 1297 LOC
**********
Sr. Sr. NOUN NOUN__Gender=Masc|Number=Sing punct Xx. False False
Presidente Presidente NOUN NOUN__Gender=Masc|Number=Sing ROOT Xxxxx True False
, , PUNCT PUNCT punct , False False
passo passar VERB VERB__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin appos xxxx True False
a o SCONJ SCONJ mark x True False
responder responder VERB VERB__VerbForm=Inf advcl xxxx True False
a o ADP ADP case x True False
este este DET DET__Gender=Masc|Number=Sing|PronType=Dem det xxxx True True
lote lotar NOUN NOUN__Gender=Masc|Number=Sing obj xxxx True False
de de ADP ADP case xx True True
três três NUM NUM__NumType=Card nummod xxxx True True
perguntas perguntar NOUN NOUN__Gender=Fem|Number=Plur nmod xxxx True False
, , PUNCT PUNCT punct , False False
as 